In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime as dt
import boto3, io, os, json, pickle, gzip
import re, pyarrow
import time
from time import gmtime, strftime
import sagemaker
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.estimator import Estimator 
from sagemaker.transformer import Transformer
from sagemaker.amazon.amazon_estimator import get_image_uri

# load a small sample dataset to local (optional)

In [3]:
"""
def load_dpc_data():
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('innovation-dev-data-ingress-bucket')
    prefix_objs = bucket.objects.filter(Prefix='EAC_D39_DPC/')
    df_dpc = []
    for obj in prefix_objs:
        if obj.key.endswith('.parquet'):
            body = obj.get()['Body'].read()
            temp = pd.read_parquet(io.BytesIO(body))
            df_dpc.append(temp.iloc[::-1])
    df_dpc_total = pd.concat(df_dpc)
    df_dpc_total.reset_index(drop=True, inplace=True)
    df_dpc_total.drop(columns=['daily_profile_coefficient_per_second','rn', 'PARTITION_YYYYMM'])
    return df_dpc_total
df_dpc_total=load_dpc_data()
"""

In [ ]:
"""
#dask version
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()
df_meter_read_2021=dd.read_parquet('s3://innovation-dev-data-ingress-bucket/METER_READINGS/2021/**/*.parquet', columns=['mpan', 'read_datetime', 'reading_type', 'register_reading', 'valid_read', 'flow_name'])
df_meter_read_2022=dd.read_parquet('s3://innovation-dev-data-ingress-bucket/METER_READINGS/2022/**/*.parquet', columns=['mpan', 'read_datetime', 'reading_type', 'register_reading', 'valid_read', 'flow_name'])
df_meter_read_2023=dd.read_parquet('s3://innovation-dev-data-ingress-bucket/METER_READINGS/2023/**/*.parquet', columns=['mpan', 'read_datetime', 'reading_type', 'register_reading', 'valid_read', 'flow_name'])
"""

In [186]:
""" the following requires pythena to be installed
s3 = boto3.client('s3')
bucket = 'innovation-dev-athena-results/tables/b922b8ff-ea60-40f6-b89b-557373790d65/'
output_location = 's3://innovation-dev-athena-results/athena-ml/'
conn = pyathena.connect(s3_staging_dir=output_location, region_name='eu-west-2',work_group='V2EngineWorkGroup')
"""

" the following requires pythena to be installed\ns3 = boto3.client('s3')\nbucket = 'innovation-dev-athena-results/tables/b922b8ff-ea60-40f6-b89b-557373790d65/'\noutput_location = 's3://innovation-dev-athena-results/athena-ml/'\nconn = pyathena.connect(s3_staging_dir=output_location, region_name='eu-west-2',work_group='V2EngineWorkGroup')\n"

In [95]:
# load training data locally
def load_eac_table(data_file):  
    s3 = boto3.resource('s3')  
    bucket = s3.Bucket('innovation-dev-athena-results')
    prefix_objs = bucket.objects.filter(Prefix='tables/'+data_file)
    df = []
    for obj in prefix_objs:
        body = obj.get()['Body'].read()
        temp = pd.read_parquet(io.BytesIO(body))
        df.append(temp)
    df_total = pd.concat(df)
    df_total.reset_index(drop=True, inplace=True)
    return df_total

In [ ]:
#df_sample_train = load_eac_table('sample training data/d88cc9bd-97a6-44f8-a36b-4e2b0ae48f17/')
df_sample_test = load_eac_table('sample training data/917e6ae0-d77e-4edf-816c-b651c26a6783/') 

In [93]:
df_sample_test.shape

(7601075, 4)

In [94]:
df_sample_test.head()

,current_eac,sum_dpc,cal_aa,previous_eac
0,2702.3,0.046784,2334.155910,2782.4
1,3703.3,0.010914,3390.028647,3719.6
2,3981.7,0.004548,3737.701509,3979.2
3,3954.7,0.099698,3861.670829,4213.4
4,7854.8,0.076180,8269.871206,7004.7


In [ ]:
df_sample_train.dtypes

In [18]:
print(df_sample_train.isna().sum(axis=0))

current_eac     0
sum_dpc         0
cal_aa          0
previous_eac    0
dtype: int64


# set up cloud environment 

In [39]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")
#xgboost_container = get_image_uri(region, 'xgboost')

#model_file_name = "xgboost"
model_file_name = "sagemaker-xgboost"

# if local model: save, upload and deploy the trained model to endpoint for inference

In [85]:
# if using xgboost api/local model then need to ensure version is the same as the aws xgboost container
#pip install xgboost=1.7.5
#import xgboost

In [3]:
#pickle.dump(xgb_model, open('model.pkl', 'wb'))
xgboost_model = pickle.load(open('model.pkl', 'rb'))

In [81]:
xgboost_model.save_model(model_file_name)

In [68]:
!tar czvf xgboost_model.tar.gz $model_file_name

xgboost


In [ ]:
bucket = "innovation-dev-models" 
prefix = "models"
fObj = open("xgboost_model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "xgboost_model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

# Sagemaker built-in model

In [93]:
# built-in model

train_data_path = f's3://innovation-dev-athena-results/tables/sample training data/d88cc9bd-97a6-44f8-a36b-4e2b0ae48f17/'
val_data_path = f's3://innovation-dev-athena-results/tables/sample training data/917e6ae0-d77e-4edf-816c-b651c26a6783/'
output_path = f's3://innovation-dev-models/sagemaker-xgb'

xgboost_estimator = Estimator(image_uri=xgboost_container, 
                    #entry_point='innovation-dev-data-insight.ipynb',
                    #framework_version="1.7-1", 
                    #hyperparameters=hyperparameters,
                    role=role,
                    instance_count=1, 
                    instance_type='ml.m5.2xlarge', 
                    #volume_size=100, # GB 
                    base_job_name = 'xgboost-training-job',
                    output_path=output_path,
                    sagemaker_session=sess)

xgboost_estimator.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:squarederror',
                        num_round=200)


train = sagemaker.inputs.TrainingInput(train_data_path, content_type="application/x-parquet")
val = sagemaker.inputs.TrainingInput(val_data_path, content_type="application/x-parquet")
xgboost_estimator.fit({'train': train, 'validation': val})


INFO:sagemaker:Creating training-job with name: xgboost-training-job-2023-05-09-14-47-17-432


2023-05-09 14:47:22 Starting - Starting the training job...
2023-05-09 14:47:35 Starting - Preparing the instances for training......
2023-05-09 14:48:57 Downloading - Downloading input data
2023-05-09 14:48:57 Training - Downloading the training image...
2023-05-09 14:49:18 Training - Training image download completed. Training in progress....[2023-05-09 14:49:41.762 ip-10-0-244-36.eu-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-05-09 14:49:41.832 ip-10-0-244-36.eu-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-05-09:14:49:42:INFO] Imported framework sagemaker_xgboost_container.training
[2023-05-09:14:49:42:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2023-05-09:14:49:42:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-09:14:49:42:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-05-09:14:49:42:INFO] Determined 0 GPU

# deploy model for inference - single model case

In [94]:
model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
#model_url = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket,key)
#model_url='s3://innovation-dev-models/sagemaker-xgb/xgboost-training-job-2023-05-05-17-41-32-982/output/model.tar.gz'
model_url='https://innovation-dev-models.s3.eu-west-2.amazonaws.com/sagemaker-xgb/xgboost-training-job-2023-05-05-17-41-32-982/output/model.tar.gz'
client = boto3.client("sagemaker")
print(model_url)

https://innovation-dev-models.s3.eu-west-2.amazonaws.com/sagemaker-xgb/xgboost-training-job-2023-05-05-17-41-32-982/output/model.tar.gz


In [95]:
primary_container = {
    "Image": xgboost_container,
    "ModelDataUrl": model_url,
}

create_model_response = client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

arn:aws:sagemaker:eu-west-2:344511226304:model/sagemaker-xgboost2023-05-09-15-13-00


In [96]:
endpoint_config_name = "xgboost-endpoint-config-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "traffic-distribution-over-models",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

xgboost-endpoint-config-2023-05-09-15-13-11
Endpoint Config Arn: arn:aws:sagemaker:eu-west-2:344511226304:endpoint-config/xgboost-endpoint-config-2023-05-09-15-13-11


In [97]:
endpoint_name = "xgboost-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name,
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

xgboost-endpoint-2023-05-09-15-13-13
arn:aws:sagemaker:eu-west-2:344511226304:endpoint/xgboost-endpoint-2023-05-09-15-13-13
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:eu-west-2:344511226304:endpoint/xgboost-endpoint-2023-05-09-15-13-13
Status: InService


# industry version EAC calculation approach

In [ ]:
#!pip install awscli
#!aws configure list
#!aws sts get-caller-identity 
#!cat container/Dockerfile

In [365]:
#pickle.dump(ind_eac, open('ind_eac.pkl', 'wb'))
#ind_eac = pickle.load(open('ind_eac.pkl', 'rb'))

#from container.multi_model.ind_eac import ind_eac 

In [431]:
%%sh

# The name of our algorithm
algorithm_name=multi-model

cd container

chmod +x multi_model/train
chmod +x multi_model/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to eu-west-2 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}
#$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

#docker build -t multi-model .
#docker tag multi-model:latest 344511226304.dkr.ecr.eu-west-2.amazonaws.com/multi-model:latest
#docker push 344511226304.dkr.ecr.eu-west-2.amazonaws.com/multi-model:latest

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}
docker push ${fullname}

Login Succeeded

Step 1/12 : FROM ubuntu:18.04
 ---> 3941d3b032a8
Step 2/12 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> f08a9b633d9d
Step 3/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 2fa6c3a6f682
Step 4/12 : RUN ln -s /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> 5465fb772d21
Step 5/12 : RUN ln -s /usr/bin/pip3 /usr/bin/pip
 ---> Using cache
 ---> 25aae49afcf3
Step 6/12 : RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn
 ---> Using cache
 ---> 528fe7a2860d
Step 7/12 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=TRUE
 ---> Using cache
 ---> f0bdb50cf0cd
Step 8/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 6967ecbed512
Step 9/12 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [432]:
account = boto3.client('sts').get_caller_identity()["Account"]
algorithm_name='multi-model'
image = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name)

#same S3 path as the other xgboost model
output_path = f's3://innovation-dev-models/multi-model/industry-version-eac-calculation'

industry_version_eac_estimator = sagemaker.estimator.Estimator(
    image_uri=image,
    role=role,
    train_instance_count=1,
    train_instance_type="ml.m5.xlarge",
    base_job_name = 'industry-version-eac-calculation-job',
    output_path=output_path,
    sagemaker_session=sess,
)

industry_version_eac_estimator.fit()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating training-job with name: industry-version-eac-calculation-job-2023-05-18-15-14-12-706


2023-05-18 15:14:13 Starting - Starting the training job...
2023-05-18 15:14:28 Starting - Preparing the instances for training...
2023-05-18 15:15:19 Downloading - Downloading input data...
2023-05-18 15:15:45 Training - Training image download completed. Training in progress.
2023-05-18 15:15:45 Uploading - Uploading generated training modelStarting the training.
/opt/program
Training complete.

2023-05-18 15:15:55 Completed - Training job completed
Training seconds: 37
Billable seconds: 37


In [ ]:
from sagemaker.predictor import csv_serializer
predictor = industry_version_eac_estimator.deploy(1, "ml.m5.xlarge", serializer=csv_serializer)

In [387]:
predictor.predict(quick_test.values).decode("utf-8")

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'2740.459138722758\n3712.405877874092\n3977.003209090916\n4143.266766582701\n7197.461855674734\n2900.7674601566523\n2979.9189625355\n3671.8175999562163\n3659.532319391288\n3497.600366891133\n'

# invoke endpoint to make inferences for both models 

In [307]:
# create a user csv file for inference
np.savetxt("test_point.csv", df_sample_test.values[:10,1:], delimiter=",") 

In [ ]:
runtime_client = boto3.client("runtime.sagemaker")

In [266]:
xgboost_endpoint_name='xgboost-endpoint-2023-05-09-16-08-43'
with open("test_point.csv", "r") as f:
    payload = f.read().strip()
response = runtime_client.invoke_endpoint(EndpointName=xgboost_endpoint_name, ContentType="text/csv", TargetModel="sagemaker-xgb.tar.gz", Body=payload)
results = response["Body"].read().decode("ascii")
print("XGBoost predicted EAC: {}".format(results))

Predicted EAC: [2753.158447265625, 3696.253173828125, 3941.913818359375, 4168.85107421875, 7261.4326171875, 2908.686279296875, 2812.234130859375, 3701.4091796875, 3686.693359375, 3512.41259765625]


In [384]:
ind_eac_endpoint_name = 'industry-version-eac-calculation-job-2023-05-17-19-35-28-146'
with open("test_point.csv", "r") as f:
    payload = f.read().strip()
response = runtime_client.invoke_endpoint(EndpointName=ind_eac_endpoint_name, ContentType="text/csv", Body=payload)
results = response["Body"].read().decode("ascii")
print("Industry version calculated EAC: {}".format(results))

Predicted EAC: 2740.459138722758
3712.405877874092
3977.003209090916
4143.266766582701
7197.461855674734
2900.7674601566523
2979.9189625355007
3671.8175999562163
3659.532319391288
3497.600366891133



# deployment for multi-container case

In [438]:
ind_eac_image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)
ind_eac_model_url = 'https://innovation-dev-models.s3.eu-west-2.amazonaws.com/multi-model/industry-version-eac-calculation/industry-version-eac-calculation-job-2023-05-18-15-14-12-706/output/model.tar.gz'
ind_eac_container = {'ContainerHostname': 'ind-eac-model',
                     'Image': "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name),
                     'ModelDataUrl': ind_eac_model_url}

xgboost_image_uri = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")
xgboost_model_url = 'https://innovation-dev-models.s3.eu-west-2.amazonaws.com/sagemaker-xgb/xgboost-training-job-2023-05-05-17-41-32-982/output/model.tar.gz'
xgboost_container = {'ContainerHostname': 'xgboost-model',
                     'Image': xgboost_image_uri,
                     'ModelDataUrl': xgboost_model_url}                  


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [439]:
#multi_container_model_file_name = 'Synthetic-EAC-multi-container-'
#multi_container_model_name = multi_container_model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
multi_container_model_name = 'Synthetic-EAC-multi-container'
client = boto3.client("sagemaker")
create_model_response = client.create_model(ModelName=multi_container_model_name,
                                            #PrimaryContainer=primary_container,
                                            Containers=[ind_eac_container, xgboost_container],
                                            InferenceExecutionConfig={'Mode': 'Direct'},
                                            ExecutionRoleArn=role)

In [440]:
#multi_container_endpoint_config_name = 'Synthetic-EAC-multi-container-endpoint-config-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
multi_container_endpoint_config_name = 'Synthetic-EAC-multi-container-endpoint-config'
create_multi_container_endpoint_config_response = client.create_endpoint_config(EndpointConfigName=multi_container_endpoint_config_name,
                                                ProductionVariants=[{#'VariantName': 'prod',
                                                                     'VariantName': 'traffic-distribution-over-models',
                                                                     'ModelName': multi_container_model_name,
                                                                     'InstanceType': 'ml.m5.xlarge',
                                                                     'InitialInstanceCount': 1,
                                                                     'InitialVariantWeight': 1,
                                                                    }] )

In [446]:
#multi_container_endpoint_name = 'Synthetic-EAC-multi-container-endpoint-config-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
multi_container_endpoint_name = 'Synthetic-EAC-multi-container-endpoint-config'
create_multi_container_endpoint_response = client.create_endpoint(EndpointName=multi_container_endpoint_name, 
                                                                  EndpointConfigName=multi_container_endpoint_config_name)
print(create_multi_container_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=multi_container_endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=multi_container_endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)
print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

arn:aws:sagemaker:eu-west-2:344511226304:endpoint/synthetic-eac-multi-container-endpoint-config
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Failed
Arn: arn:aws:sagemaker:eu-west-2:344511226304:endpoint/synthetic-eac-multi-container-endpoint-config
Status: Failed


In [ ]:
# create a user csv file for inference
np.savetxt("test_point.csv", df_sample_test.values[:10,1:], delimiter=",") 
with open("test_point.csv", "r") as f:
    payload = f.read().strip()

In [ ]:
runtime_client = boto3.client("runtime.sagemaker")
xgboost_response = runtime_client.invoke_endpoint(EndpointName=multi_container_endpoint_name, 
                                          ContentType="text/csv", 
                                          EndpointName="mnist-multi-container-ep",
                                          #ContentType="application/json",
                                          #Accept="application/json",
                                          #Body=json.dumps({"instances": np.expand_dims(tf_samples, 3).tolist()})
                                          TargetContainerHostname="xgboost-model",
                                          #TargetModel="sagemaker-xgb.tar.gz", 
                                          Body=payload)

ind_eac_response = runtime_client.invoke_endpoint(EndpointName=multi_container_endpoint_name, 
                                          ContentType="text/csv", 
                                          EndpointName="mnist-multi-container-ep",
                                          #ContentType="application/json",
                                          #Accept="application/json",
                                          #Body=json.dumps({"inputs": np.expand_dims(tf_samples, 3).tolist()})
                                          TargetContainerHostname="ind-eac-model",
                                          #TargetModel="sagemaker-xgb.tar.gz", 
                                          Body=payload)

xgboost_results = xgboost_response["Body"].read().decode("ascii")
ind_eac_results = ind_eac_response["Body"].read().decode("ascii")
print("XGBoost predicted EAC: {}".format(xgboost_results))
print("Industry-version equation calculated EAC: {}".format(ind_eac_results))

# deployment for multi-model case (obsoleted)
#only works for model with same frameworks

In [40]:
from sagemaker.multidatamodel import MultiDataModel

In [ ]:
ind_eac_image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)
ind_eac_model_url = 'https://innovation-dev-models.s3.eu-west-2.amazonaws.com/multi-model/industry-version-eac-calculation/industry-version-eac-calculation-job-2023-05-18-15-14-12-706/output/model.tar.gz'
ind_eac_container = {'ContainerHostname': 'ind-eac-model',
                     'Image': "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name),
                     'ModelDataUrl': ind_eac_model_url}

xgboost_image_uri = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")
xgboost_model_url = 'https://innovation-dev-models.s3.eu-west-2.amazonaws.com/sagemaker-xgb/xgboost-training-job-2023-05-05-17-41-32-982/output/model.tar.gz'
xgboost_container = {'ContainerHostname': 'xgboost-model',
                     'Image': xgboost_image_uri,
                     'ModelDataUrl': xgboost_model_url}  



model_data_url='s3://innovation-dev-models/multi-model/'
multi_model_container = {"Image": ind_eac_image_uri,
             "ModelDataUrl": model_data_url, 
             "Mode": "MultiModel"}

In [ ]:
multi_model_model_name = 'Synthetic-EAC-multi-model'
create_model_response = client.create_model(ModelName=multi_model_model_name, 
                                            ExecutionRoleArn=role, 
                                            PrimaryContainer=multi_model_container)
print("Model Arn: " + create_model_response["ModelArn"])

In [ ]:
multi_model_endpoint_config_name = 'Synthetic-EAC-multi-model-endpoint-config'
create_multi_model_eendpoint_config_response = client.create_endpoint_config(EndpointConfigName=multi_model_endpoint_config_name,
                                                                             ProductionVariants=[
                                                                                 {"InstanceType": "ml.m5.xlarge",
                                                                                  "InitialVariantWeight": 1,
                                                                                  "InitialInstanceCount": 1,
                                                                                  "ModelName": multi_model_model_name,
                                                                                  "VariantName": "AllTraffic"}])
print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
multi_model_endpoint_name = 'Synthetic-EAC-multi-container-endpoint-config'
create_multi_model_endpoint_response = client.create_endpoint(EndpointName=multi_model_endpoint_name, 
                                                              EndpointConfigName=multi_model_endpoint_config_name)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])


resp = client.describe_endpoint(EndpointName=multi_model_endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=multi_model_endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)
print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [184]:
sagemaker_xgb = xgboost_estimator.create_model(role=role, image_uri=xgboost_container)
endpoint_name = "xgboost-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [190]:
mme = MultiDataModel(
    name=model_name,
    model_data_prefix=f's3://innovation-dev-models/multi-model/',
    model=sagemaker_xgb,
    sagemaker_session=sess
)
predictor = mme.deploy(
    initial_instance_count=1, instance_type='ml.m5.xlarge', endpoint_name=endpoint_name
)

INFO:sagemaker:Creating model with name: sagemaker-xgboost2023-05-09-15-13-00
INFO:sagemaker:Creating endpoint-config with name xgboost-endpoint-2023-05-09-16-08-43
INFO:sagemaker:Creating endpoint with name xgboost-endpoint-2023-05-09-16-08-43


----!

In [194]:
list(mme.list_models())

['', 'sagemaker-xgb.tar.gz']

In [192]:
artifact_path = xgboost_estimator.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]
model_name = artifact_path.split("/")[-4] + ".tar.gz"
# This is copying over the model artifact to the S3 location for the MME.
mme.add_model(model_data_source=artifact_path, model_data_path=model_name)

's3://innovation-dev-models/multi-model/sagemaker-xgb.tar.gz'

# batch transformation on S3 bucket data with trained model (obsolete)

In [22]:
# serialization and deserialization for mode handler to accept parquet format for batch transform jobs
from io import BytesIO
from typing import BinaryIO
import pandas as pd
from botocore.response import StreamingBody
def input_fn(
  serialized_input_data: StreamingBody,
  content_type: str = "application/x-parquet",
) -> pd.DataFrame:
  """Deserialize inputs"""
  if content_type == "application/x-parquet":
    data = BytesIO(serialized_input_data)
    df = pd.read_parquet(data)
    return df
  else:
    raise ValueError(
      "Expected `application/x-parquet`."
    )

def output_fn(output: pd.DataFrame, accept: str = "application/x-parquet") -> BinaryIO:
  """Model output handler"""
  if accept == "application/x-parquet":
    buffer = BytesIO()
    output.to_parquet(buffer)
    
    return buffer.getvalue()
  else:
    raise Exception("Requested unsupported ContentType in Accept: " + accept)


In [42]:
# batch transformation from the following 
batch_input_path = 's3://innovation-dev-athena-results/tables/bacth transformation/db716314-24db-4b3f-9401-65db2c6edb6d/'
batch_output_path = 's3://innovation-dev-athena-results/tables/bacth transformation/batch output/'
xgboost_transformer = xgboost_estimator.transformer(instance_count=1,
                          instance_type='ml.m5.4xlarge',
                          output_path=batch_output_path,
                          accept='application/x-parquet',
                          strategy='MultiRecord')

# calls that object's transform method to create a transform job
xgboost_transformer.transform(data=batch_input_path, content_type='application/x-parquet')
transformer.wait()



INFO:sagemaker:Creating model with name: xgboost-training-job-2023-05-04-09-57-27-635
INFO:sagemaker:Creating transform job with name: xgboost-training-job-2023-05-04-09-57-28-324


..........................[2023-05-04:10:01:44:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-04:10:01:44:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-04:10:01:44:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2023-05-04:10:01:44:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-04:10:01:44:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-04:10:01:44:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|exe

UnexpectedStatusException: Error for Transform job xgboost-training-job-2023-05-04-09-57-28-324: Failed. Reason: ClientError: See job logs for more information

In [ ]:
batch_output = 's3://innovation-dev-athena-results/tables/batch-output.out' 
batch_output = pd.read_csv(batch_output, header=None, encoding = "ISO-8859-1") # header = none